In [12]:
import numpy as np
import pandas as pd
import matplotlib 
from matplotlib import pyplot as plt

# Imports
import pandas as pd

import spacy
from spacy.tokenizer import Tokenizer
from spacy.lang.es import Spanish

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

import pickle

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

# https://www.aurigait.com/blog/recommendation-system-using-knn/
# https://www.analyticsvidhya.com/blog/2020/08/recommendation-system-k-nearest-neighbors/
# https://www.mygreatlearning.com/blog/nltk-tutorial-with-python/
# https://www.datacamp.com/community/tutorials/recommender-systems-python

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [5]:
file = open("Documentos.txt", "r", encoding='utf8')
lines = file.readlines()
df = pd.DataFrame(lines, columns=['Documentos'])

df_copy = df.copy()
for i,x in enumerate(df_copy['Documentos']):
        df_copy['Documentos'][i] = df_copy['Documentos'][i][:-2]
        print(df_copy['Documentos'][i])

df['Documentos'] = df_copy['Documentos']

df.head()

Los grupos musculares se ejercitan en función del ejercicio específico que se realice, por ejemplo, si quieres realizar un ejercicio de alta intensidad, optarás por uno anaeróbico y viceversa
Existen dos tipos de actividades física que se suelen practicar, los ejercicios aeróbicos y los ejercicios anaeróbicos
Los deportes más practicados a nivel mundial son el fútbol y el baloncesto, donde se fomenta el compañerismo y el respeto
Practicar deporte es altamente recomendado para mejorar la salud física y mental, según han publicado varias revistas y organismos
Las películas con más Oscars de la historia son Titanic, BenHur y El Señor de los Anillos
Se han realizado varios estudios reflejando la importancia del ejericio como medida para contrarrestar el estrés


,Documentos
0,Los grupos musculares se ejercitan en función ...
1,Existen dos tipos de actividades física que se...
2,Los deportes más practicados a nivel mundial s...
3,Practicar deporte es altamente recomendado par...
4,Las películas con más Oscars de la historia so...


In [35]:
df['Documentos'] = df['Documentos'].apply(lambda x: str.lower(x))

In [36]:
# https://www.mygreatlearning.com/blog/nltk-tutorial-with-python/
    
### Primero tokenizamos los 4 primeros documentos
def tokenize_words(text):
    tokenized_text=word_tokenize(text)
    print(tokenized_text)
    print(type(tokenized_text))
    
    return tokenized_text

# df_tokenized = df.copy()
df['Words_tokenized'] = df['Documentos'].apply(lambda x: tokenize_words(x))
    


['los', 'grupos', 'musculares', 'se', 'ejercitan', 'en', 'función', 'del', 'ejercicio', 'específico', 'que', 'se', 'realice', ',', 'por', 'ejemplo', ',', 'si', 'quieres', 'realizar', 'un', 'ejercicio', 'de', 'alta', 'intensidad', ',', 'optarás', 'por', 'uno', 'anaeróbico', 'y', 'viceversa']
<class 'list'>
['existen', 'dos', 'tipos', 'de', 'actividades', 'física', 'que', 'se', 'suelen', 'practicar', ',', 'los', 'ejercicios', 'aeróbicos', 'y', 'los', 'ejercicios', 'anaeróbicos']
<class 'list'>
['los', 'deportes', 'más', 'practicados', 'a', 'nivel', 'mundial', 'son', 'el', 'fútbol', 'y', 'el', 'baloncesto', ',', 'donde', 'se', 'fomenta', 'el', 'compañerismo', 'y', 'el', 'respeto']
<class 'list'>
['practicar', 'deporte', 'es', 'altamente', 'recomendado', 'para', 'mejorar', 'la', 'salud', 'física', 'y', 'mental', ',', 'según', 'han', 'publicado', 'varias', 'revistas', 'y', 'organismos']
<class 'list'>
['las', 'películas', 'con', 'más', 'oscars', 'de', 'la', 'historia', 'son', 'titanic', ','

In [31]:
stopword_es

['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con',
 'no',
 'una',
 'su',
 'al',
 'lo',
 'como',
 'más',
 'pero',
 'sus',
 'le',
 'ya',
 'o',
 'este',
 'sí',
 'porque',
 'esta',
 'entre',
 'cuando',
 'muy',
 'sin',
 'sobre',
 'también',
 'me',
 'hasta',
 'hay',
 'donde',
 'quien',
 'desde',
 'todo',
 'nos',
 'durante',
 'todos',
 'uno',
 'les',
 'ni',
 'contra',
 'otros',
 'ese',
 'eso',
 'ante',
 'ellos',
 'e',
 'esto',
 'mí',
 'antes',
 'algunos',
 'qué',
 'unos',
 'yo',
 'otro',
 'otras',
 'otra',
 'él',
 'tanto',
 'esa',
 'estos',
 'mucho',
 'quienes',
 'nada',
 'muchos',
 'cual',
 'poco',
 'ella',
 'estar',
 'estas',
 'algunas',
 'algo',
 'nosotros',
 'mi',
 'mis',
 'tú',
 'te',
 'ti',
 'tu',
 'tus',
 'ellas',
 'nosotras',
 'vosotros',
 'vosotras',
 'os',
 'mío',
 'mía',
 'míos',
 'mías',
 'tuyo',
 'tuya',
 'tuyos',
 'tuyas',
 'suyo',
 'suya',
 'suyos',
 'suyas',
 'nuestro',
 'nuestra',
 'nuestros',
 'nuestras',
 'vuestro'

In [37]:
stopword_es = nltk.corpus.stopwords.words('spanish')

def remove_stopwords(text):
    text = [word for word in text if word not in stopword_es]
    return text

df['Sin_stopwords'] = df['Words_tokenized'].apply(lambda x: remove_stopwords(x))

In [38]:
df.head()

,Documentos,Words_tokenized,Sin_stopwords
0,los grupos musculares se ejercitan en función ...,"[los, grupos, musculares, se, ejercitan, en, f...","[grupos, musculares, ejercitan, función, ejerc..."
1,existen dos tipos de actividades física que se...,"[existen, dos, tipos, de, actividades, física,...","[existen, dos, tipos, actividades, física, sue..."
2,los deportes más practicados a nivel mundial s...,"[los, deportes, más, practicados, a, nivel, mu...","[deportes, practicados, nivel, mundial, fútbol..."
3,practicar deporte es altamente recomendado par...,"[practicar, deporte, es, altamente, recomendad...","[practicar, deporte, altamente, recomendado, m..."
4,las películas con más oscars de la historia so...,"[las, películas, con, más, oscars, de, la, his...","[películas, oscars, historia, titanic, ,, benh..."


In [43]:


#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(lowercase = False)

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df['Sin_stopwords'][0])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(20, 16)

In [44]:
tfidf_matrix

<20x16 sparse matrix of type '<class 'numpy.float64'>'
	with 17 stored elements in Compressed Sparse Row format>